In [1]:
from __future__ import absolute_import, division, print_function

import os
import pickle
from six.moves import urllib

import tflearn
from tflearn.data_utils import *
from tflearn.layers import *

import gensim
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.word2vec import Word2Vec


path = "data/nenovinite.txt"
char_idx_file = 'char_idx.pickle'


In [2]:
maxlen = 25

char_idx = None
if os.path.isfile(char_idx_file):
  print('Loading previous char_idx')
  char_idx = pickle.load(open(char_idx_file, 'rb'))

X, Y, char_idx = \
    textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3, pre_defined_char_idx=char_idx)

Loading previous char_idx
Vectorizing text...
Text total length: 101,436
Distinct chars   : 147
Total sequences  : 33,804


In [4]:
pickle.dump(char_idx, open(char_idx_file,'wb'))

g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.gru(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.gru(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.gru(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
learning_rate=0.001)

In [5]:
model = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='nenovini2')
model.load("data/bgnews3")

INFO:tensorflow:Restoring parameters from /home/alex/fmi/ML/project/data/bgnews3


In [6]:
for i in range(40):
    seed = random_sequence_from_textfile(path, maxlen)
    print(seed)
    model.fit(X, Y, validation_set=0.1, batch_size=128, n_epoch=1)
    print("-- TESTING...")
    print("-- Test with temperature of 1.0 --")
    print(model.generate(600, temperature=1.0, seq_seed=seed))
    print("-- Test with temperature of 0.5 --")
    print(model.generate(600, temperature=0.5, seq_seed=seed))
    model.save("data/bgnews3")
print(model.generate(600, temperature=0.5, seq_seed="оставка Бойко Борисов"))
print(model.generate(600, temperature=0.5, seq_seed="България политика оставка"))

Training Step: 10709  | total loss: 3.23745 | time: 232.600s
| Adam | epoch: 040 | loss: 3.23745 -- iter: 30336/30423
Training Step: 10710  | total loss: 3.23589 | time: 242.853s
| Adam | epoch: 040 | loss: 3.23589 | val_loss: 3.19920 -- iter: 30423/30423
--
INFO:tensorflow:/home/alex/fmi/ML/project/nenovini2-10710 is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
-- TESTING...
-- Test with temperature of 1.0 --
 на Наглите през погледитатаи"росивРвиирбоч -с1и еАнаит танСа.
рк хии.вде вемит д "и ди оиебзарноИтап я 
т лл нуочаеаи4вт рд"кобнро ,вии таДвtлрвДи и щяркуагакдеввтезадт  e с
 яот
лавм оарочзрспн сп

In [7]:
model.save("data/bgnews3")

INFO:tensorflow:/home/alex/fmi/ML/project/data/bgnews3 is not in all_model_checkpoint_paths. Manually adding it.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
